In [3]:
'''!pip install -U spacy
!pip install spacy_transformers
!pip install --upgrade pydantic'''
!pip install spacy-transformers


# install any necessary packages
#!pip install fitz      @D
#!pip install PyMuPDF   @D

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
# Change the working directory to the project folder




In [7]:
spacy.__version__

'3.7.2'

In [4]:
with open('/Users/ekamsingh/pythonProject2/generated_resumes.json', 'r') as file:
    data = file.read()
import json

try:
    data = json.loads(data)
except json.JSONDecodeError as e:
    print(f"Error: {e}")


Error: Expecting ',' delimiter: line 47002 column 5 (char 3355791)


In [5]:
#Loading the annotated data
cv_data=json.load(open('/Users/ekamsingh/pythonProject2/generated_resumes.json'))

#Number of items in the dataset 
len(cv_data)

#Show first item of the data
print(cv_data[0])

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [6]:
!python -m spacy init fill-config /Users/ekamsingh/Downloads/base_config_2.cfg  /Users/ekamsingh/Downloads/config2.cfg

/opt/anaconda3/envs/hello/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/hello/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/Users/ekamsingh/Downloads/config2.cfg
You can now add your data and train your pipeline:
python -m spacy train config2.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [84]:
# DocBin objects in spaCy are a serialized format for storing multiple Doc objects efficiently. 
#They are used to save and load annotated documents in a binary format, which is particularly useful for training and processing pipelines in spaCy.

#Key Features and Importance of DocBin Objects:
#Efficiency:

#DocBin objects store data in a compressed binary format, which reduces the file size and speeds up loading and saving compared to text formats like JSON.
#Compatibility:

#They are compatible with spaCy's pipeline, making it easy to integrate them into various stages of data processing and model training.
#Data Integrity:

#By using DocBin objects, you ensure that the data structure and annotations are preserved accurately, reducing the chances of data corruption or format inconsistencies.

In [85]:
# function to create spacy DocBIn objects from the annotated data 

In [7]:
# Define a function to create spaCy DocBin objects from the annotated data

def get_spacy_doc(file, data):
    nlp=spacy.blank("en")
    db=DocBin()

    #Iterate thorugh the data 
    for text, annot in  tqdm(data):        #This is a standard for-loop in Python that iterates over each item in data. Each item is expected to be a tuple containing text and annot.
        doc=nlp.make_doc(text)             #tqdm is a Python library that provides a fast, extensible progress bar for loops and other iterable operations. Wrapping data with tqdm(data) will display a progress bar that updates with each iteration of the loop, giving you a visual indication of how much of the loop has been completed.
        annot=annot['entities']            #To pass the enitties to the data 


        ents=[]                            #all the different entities 
        entity_indices=[]                  #all the emntiy indices 


        #extract entities from annotation 
        for start,end , label in annot:
            skip_entity = False
            for idx in range(start , end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue


            entity_indices=entity_indices + list(range(start, end))
            try :
                span=doc.char_span(start , end , label=label , alignment_mode="strict") #In spaCy, a Span is a sequence of tokens in a Doc object. It's a convenient way to refer to and manipulate a contiguous slice of the text.
            except:
                continue

            if span is None:                                                            #  ERROR FUNCTION 
            # Log errors for annotations that couldn't be processed
                err_data = str([start, end]) + "    " + str(text) + "\n"
                file.write(err_data)
            else:
                ents.append(span)

        try:
            doc.ents=ents
            db.add(doc)
        except:
            pass

    return db 

In [87]:
# Data spliiting into training and testing sets and create Spacy DocBin objects for them 
# Also log errors for the annotation process 

In [8]:
# Split the annotated data :
from sklearn.model_selection import train_test_split

train , test = train_test_split(cv_data , test_size = 0.2)

#Display the number of items in training and testing dataset
len(train) , len(test)

file=open('/Users/ekamsingh/resume_p/error.txt','w')

# Spacy DocBin objects for training and testing data 

db = get_spacy_doc(file, train)
db.to_disk('/Users/ekamsingh/resume_p/train_data.spacy')

db = get_spacy_doc(file , test)
db.to_disk('/Users/ekamsingh/resume_p/test_data.spacy')

# close the log error  file 
file.close()

/opt/anaconda3/envs/hello/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/hello/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/hello/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
100%|██████████| 240/240 [00:00<00:00, 319.14it/s]


In [89]:
# Model Training 
# Train a NER model with the provided configuration

In [30]:
import torch
import numpy as np 
import pandas as pd 
import sklearn
import matplotlib.pyplot as plt
print(f"PyTorch version: {torch.__version__}")
# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")
# Set the device
device = "mps" if torch. backends.mps. is_available() else "chi"
print(f"Using device: {device}")

!pip install torch torchvision torchaudio 

PyTorch version: 2.3.1
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.5 MB/s eta 0:00:0000:0100:01


In [9]:
!python -m spacy train /Users/ekamsingh/Downloads/config2.cfg --output /Users/ekamsingh/resume_p/ouput6 --paths.train /Users/ekamsingh/resume_p/train_data.spacy --paths.dev /Users/ekamsingh/resume_p/test_data.spacy 


'''/Users/ekamsingh/EY - Internship Project/config/config.cfg :
This is the path to the configuration file. 
The configuration file (config.cfg) contains all the settings and hyperparameters needed for training the model, including the components of the pipeline, optimization parameters, and data paths.'''

'''--output /content/drive/MyDrive/Custom_NER/trained_models/output:
This specifies the directory where the trained model and other output files will be saved.
After training, this directory will contain the trained model files, logs, and other relevant outputs.'''

''' --paths.train /content/drive/MyDrive/Custom_NER/trained_models/train_data.spacy:
This specifies the path to the training data in the .spacy format. 
The DocBin object for the training data, which was created and saved earlier, will be used during the training process.'''

'''--paths.dev /content/drive/MyDrive/Custom_NER/trained_models/test_data.spacy:
This specifies the path to the development (or validation) data in the .spacy format.
The DocBin object for the test data will be used to evaluate the model's performance during training.'''

ℹ Saving to output directory: /Users/ekamsingh/resume_p/ouput6
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
/opt/anaconda3/envs/hello/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/hello/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/hello/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`

"--paths.dev /content/drive/MyDrive/Custom_NER/trained_models/test_data.spacy:\nThis specifies the path to the development (or validation) data in the .spacy format.\nThe DocBin object for the test data will be used to evaluate the model's performance during training."

In [3]:
nlp=spacy.load('/Users/ekamsingh/resume_p/model_for_ner_saved6')

/opt/anaconda3/envs/hello/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/hello/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/hello/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/hello/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _

In [11]:
# Saving the model 

#Assuming nlp is your trained model
output_dir = '/Users/ekamsingh/resume_p/model_for_ner_saved6'
nlp.to_disk(output_dir)

print(f"Model saved to {output_dir}")

Model saved to /Users/ekamsingh/resume_p/model_for_ner_saved6


In [ ]:
#testing the dataset


In [ ]:
#load the trained nlp model 


import sys
import fitz

#fname='/Users/ekamsingh/custom_ner/resume_ekam _singh3.pdf'
fname= '/Users/ekamsingh/Downloads/resumes/RishabhGaur.pdf'

#Open the document using PyMupdf

doc=fitz.open(fname)

#initialize an empty string to store extracted text from the PDF

text=''

#iterate thorugh each page and concatenate the text 

for page in doc:
    text=text + str(page.get_text()) #extracting text from each page of the doc 

text=text.strip() #removes any leading or trailing whitespace 

#Display the extracted text 

print(text)

In [6]:
import re

# Define the resume text

# Define the resume text
resume_text = """
Rishabh Gaur
E-mail: 11seprishabh@gmail.com
Phone no: +91-9660600736
Summary
 2.5 year of experience in I.T. industry in web and mobile development.
 Strong team player with sound experience in all phases of SDLC.
 Experience in Analyzing and Documenting the Customer requirements, Design,
Development, Quality adherence, Implementation and Documentation
o Good communication, collaboration & team building skills with proficiency at
grasping new technical concepts quickly and utilize the same in a productive
manner.
 My technological forte is:
o Operating Systems: Ubuntu, Windows and macOS.
o Technologies & Skill sets: Ionic, AngularJS (v: 6), NodeJS, JavaScript, IOS, Nginx,
and Android.
o Databases: MySQL and MongoDB.
o Tools: Visual Studio Code and Postman.
o Version Control Systems: GitHub.
Education
 Bachelors of Technology - 2017 –JECRC, Jaipur - Information Technology: 73%
 XII - 2013 – RBSE: 80.60%
 X - 2011 – RBSE: 80.50%
Experience
 May, 2016 - July, 2016
Android Developer (Intern) | ComSkyNet technology| Jaipur
 Feb, 2017-May, 2019
Software Engineer (Full stack developer) | Newgen Software Technologies Ltd. | Noida
 June, 2020 – Jan, 2022
Freelancing
Projects
 WORKEEFY: A innovative digital marketplace for instant home services provided by
trained professionals that serve clients throughout the Toronto, GTA, Ottawa and now
entering Calgary region too. Also, we had developed an admin side panel to handle all the
flow of user Workeefyer (service provider) and admin activities.
Tools Used: Visual Studio Code.
Technologies: IonicJS, AngularJS, NodeJS.
Team Size: 6
 I-Aptt: It is an Android app which is useful for the students who are preparing for various
competitive exams like GATE, IAS and GRE etc.
Tools Used: Visual Studio Code, Android Studio.
Technologies: firebase.
Team Size: 1
 Blood-Donor Family: It is an Android application which can easily find blood donor, when
immediate need of donor occurs. Using location we can easily locate donor and having an
interactive interface to interact with each other.
Tools Used: Visual Studio Code, Postman, Android Studio.
Technologies: firebase, Google cloud messaging.
Team Size: 1
Awards and Acknowledgments
 Completed the Community Coaches Training Program of Sports & Games for the Disabled
by Ministry of Youth Affairs & Sports, Gov. of India.
 Gold Medalist in “Talent Search, Culture & General knowledge” A State level Competition
by Vidhyarthi Kalyan trust, Jaipur.
 Awarded by Scholarship of MHRD (Ministry of Human Resource Development).
Extra-Curricular Activities
 Core team member of Renaissance’16-An annual techno cultural fest of JECRC, Jaipur.
 Volunteered “RUNWAY TO SUCCESS” an entrepreneurship event of TIE (The Indus
Entrepreneurs), Jaipur.
 Coordinate an event “New Venture” a business plan competition.
 Participated as an International Press member in JECRC MUN’16.
 Member of “Fotogra Freaks”, photography club of JECRC, Jaipur.
 Student Coordinator at EDC-IIPC cell at JECRC, Jaipur.
 Organized various events during News Week 2017-2019 in Newgen Software.
 Working as a journalist in weekly newspaper “Insaaf Ki Dagar”.
Seminars/Workshop attended
 Attended 2 days’ workshop on “Entrepreneurship Development Program” by MANASSO
(Management associate private limited, Noida).
 Actively participated in Food Tech (2014-15, 2015-16) organized by MSME.
 Attended 1-week workshop on Exports Marketing Documentation & Procedure by MSME
(Ministry of Micro, small & Medium Enterprises).
 Attended 2 days’ workshop “Digital India Summit” by Microsoft
"""

# Define the sections to remove
sections_to_remove = [
    "Projects",
    "Awards and Acknowledgments",
    "Extra-Curricular Activities",
    "Seminars/Workshops Attended"
]

# Function to remove sections from resume
def remove_sections(text, sections):
    for section in sections:
        section_pattern = re.compile(rf"{section}.*?(?=\n\S|\Z)", re.DOTALL)
        text = re.sub(section_pattern, "", text)
    return text

# Remove specified sections
cleaned_resume_text = remove_sections(resume_text, sections_to_remove)

# Print the cleaned resume
print(cleaned_resume_text)


Rishabh Gaur
E-mail: 11seprishabh@gmail.com
Phone no: +91-9660600736
Summary
 2.5 year of experience in I.T. industry in web and mobile development.
 Strong team player with sound experience in all phases of SDLC.
 Experience in Analyzing and Documenting the Customer requirements, Design,
Development, Quality adherence, Implementation and Documentation
o Good communication, collaboration & team building skills with proficiency at
grasping new technical concepts quickly and utilize the same in a productive
manner.
 My technological forte is:
o Operating Systems: Ubuntu, Windows and macOS.
o Technologies & Skill sets: Ionic, AngularJS (v: 6), NodeJS, JavaScript, IOS, Nginx,
and Android.
o Databases: MySQL and MongoDB.
o Tools: Visual Studio Code and Postman.
o Version Control Systems: GitHub.
Education
 Bachelors of Technology - 2017 –JECRC, Jaipur - Information Technology: 73%
 XII - 2013 – RBSE: 80.60%
 X - 2011 – RBSE: 80.50%
Experience
 May, 2016 - July, 2016
Android Develop

In [7]:
'''bullet_points = [
    "•",  # Solid bullet
    "-",  # Dash
    "–",  # En dash
    "—",  # Em dash
    "○",  # Hollow circle
    "▪",  # Solid square
    "■",  # Filled square
    "▶",  # Filled triangle
    "►",  # Solid arrowhead
    "✓",  # Checkmark
    "➤",  # Arrowhead
    "‣",  # Triangular bullet
    "●",
    
]

# Remove bullet points from text
for bullet in bullet_points:
    text = cleaned_resume_text.replace(bullet, '')

print(text)'''


Rishabh Gaur
E-mail: 11seprishabh@gmail.com
Phone no: +91-9660600736
Summary
 2.5 year of experience in I.T. industry in web and mobile development.
 Strong team player with sound experience in all phases of SDLC.
 Experience in Analyzing and Documenting the Customer requirements, Design,
Development, Quality adherence, Implementation and Documentation
o Good communication, collaboration & team building skills with proficiency at
grasping new technical concepts quickly and utilize the same in a productive
manner.
 My technological forte is:
o Operating Systems: Ubuntu, Windows and macOS.
o Technologies & Skill sets: Ionic, AngularJS (v: 6), NodeJS, JavaScript, IOS, Nginx,
and Android.
o Databases: MySQL and MongoDB.
o Tools: Visual Studio Code and Postman.
o Version Control Systems: GitHub.
Education
 Bachelors of Technology - 2017 –JECRC, Jaipur - Information Technology: 73%
 XII - 2013 – RBSE: 80.60%
 X - 2011 – RBSE: 80.50%
Experience
 May, 2016 - July, 2016
Android Develop

In [11]:
text = ' '.join(cleaned_resume_text.split())  #splits the string with extr whitespace as a delimeter and then after splitting it joins the string again
print(text)

Rishabh Gaur E-mail: 11seprishabh@gmail.com Phone no: +91-9660600736 Summary  2.5 year of experience in I.T. industry in web and mobile development.  Strong team player with sound experience in all phases of SDLC.  Experience in Analyzing and Documenting the Customer requirements, Design, Development, Quality adherence, Implementation and Documentation o Good communication, collaboration & team building skills with proficiency at grasping new technical concepts quickly and utilize the same in a productive manner.  My technological forte is: o Operating Systems: Ubuntu, Windows and macOS. o Technologies & Skill sets: Ionic, AngularJS (v: 6), NodeJS, JavaScript, IOS, Nginx, and Android. o Databases: MySQL and MongoDB. o Tools: Visual Studio Code and Postman. o Version Control Systems: GitHub. Education  Bachelors of Technology - 2017 –JECRC, Jaipur - Information Technology: 73%  XII - 2013 – RBSE: 80.60%  X - 2011 – RBSE: 80.50% Experience  May, 2016 - July, 2016 Android Develope

In [9]:
doc=nlp(text)

for ent in doc.ents:
    print(ent.text , "---->" ,ent.label_)

Rishabh Gaur ----> NAME
11seprishabh@gmail.com ----> EMAIL ADDRESS
2.5 year ----> YEARS OF EXPERIENCE
Bachelors of Technology ----> DEGREE
2017 ----> GRADUATION YEAR
2013 ----> GRADUATION YEAR
Jaipur ----> COMPANIES WORKED AT


In [ ]:
from spacy import displacy

displacy.serve(doc, style="ent",auto_select_port=True)

/opt/anaconda3/envs/hello/lib/python3.9/site-packages/spacy/util.py:1817: UserWarning: [W124] 0.0.0.0:5000 is already in use, using the nearest available port 5003 as an alternative.
  warnings.warn(Warnings.W124.format(host=host, port=start, serve_port=port))
/opt/anaconda3/envs/hello/lib/python3.9/site-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5003 ...



In [ ]:
# confusion matrix to avoid clashing 